<a href="https://colab.research.google.com/github/roberttnovak/MedicineScraper/blob/main/playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medicine Scraper

<img src="https://cima.aemps.es/cima/resources/images/CimaTranspaPeq.gif">


In [ ]:
# Install dependencies
!apt update
!apt install chromium-chromedriver
!pip install selenium

In [ ]:
# Import modules
import re
from time import sleep
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By #Para poder usar By. dentro de find_element
from selenium.webdriver.support.ui import WebDriverWait #Para esperar hasta cargar la página 
from selenium.webdriver.support import expected_conditions as EC #Para esperar hasta cargar la página 


# Set driver options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # No interface
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument('--disable-dev-shm-usage')

## **Código base**


In [ ]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get('https://cima.aemps.es/cima/publico/home.html')
driver.find_element(By.CSS_SELECTOR,"button[onclick='biosimilaresTodos()']") # Example search in the homepage

# Navigate to "listado de búsqueda"
buscador = driver.find_element(By.ID, "inputbuscadorsimple")
buscador.clear()
buscador.send_keys("*")
buscador.send_keys(Keys.ENTER)
sleep(3)  # If we do not wait, the `page_source` will not have time to change and it will be showed the previous page (the homepage in this case)
driver.title

'.:: CIMA ::. Resultados de la búsqueda de medicamentos'

In [ ]:
# En la página aparecen una serie de filtros por defecto. 
# Nos aseguramos de que no se aplique ningún filtro comprobando, en primer lugar, cuáles son los filtros aplicados por defecto
# y, en segundo lugar, deseleccionandolos. De esta forma, obtenemos todos los medicamentos 

from selenium.common.exceptions import NoSuchElementException
filtros_busqueda = {
    "filtroRecetaSi"              :  "filtro de los medicamentos con receta"                      ,
    "filtroRecetaNo"             :  "filtro de los medicamentes sin receta"                      ,                                    
    "filtroTrianguloSi"          :  "filtro de los medicamentos con seguimiento adicional"       ,
    "filtroTrianguloNo"          :  "filtro de los medicamentos sin seguimiento adicional"       ,
    "filtroHuerfanoSi"           :  "filtro de los medicamentos que son huérfanos"               ,
    "filtroHuerfanoNo"           :  "filtro de los medicamentos que no son huérfanos"            ,
    "filtroBiosimilarSi"         :  "filtro de los medicamentos que son biosimilares"            ,
    "filtroBiosimilarNo"         :  "filtro de los medicamentos que no son biosimilares"         ,
    "filtroComercializadoSi"     :  "filtro de los medicamentos comercializados"                 ,
    "filtroComercializadoNo"     :  "filtro de los medicamentos no comercializados"              ,
    "filtroImpParalelasSi"       :  "filtro de los medicamentos con importación paralela"        ,
    "filtroImpParalelasNo"       :  "filtro de los medicamentos sin importación paralela"        ,
    "filtroAutorizado"           :  "filtro de los medicamentos autorizados"                     ,
    "filtroSuspendido"           :  "filtro de los medicamentos suspendidos"                     ,
    "filtroRevocado"             :  "filtro de los medicamentos revocados"                       ,
    "filtroBiologicos"           :  "filtro de los medicamentos biológicos"                      ,
    "filtroPactivos"             :  "filtro de los medicamentos con estrecho margen terapéutico" ,
    "filtroApRespiratorio"       :  "filtro de los medicamentos por vía respiratoria"                                   
}
for i,v in filtros_busqueda.items():
    try:  #Nos aseguramos de que el código se pueda ejecutar si salta algún error
        if not driver.find_element(By.ID, i).is_selected():
            checkbox=driver.find_element(By.CSS_SELECTOR, "label[for={}]".format(i))
            driver.execute_script("arguments[0].click();", checkbox)
    except NoSuchElementException: #Identificamos el error correspondiente al de los elementos que identifican los checkbox
        print("El {} ya no existe en la página web o su id en el html ha cambiado de nombre".format(v))
    except:
        print("Ha ocurrido un error para el {}".format(v))

In [ ]:
# References: https://devqa.io/selenium-css-selectors/ 
# Search a specific medicine
# med_81012 = driver.find_element(By.CSS_SELECTOR, "div[onclick=\"medicamentoOnSelect('81012')\"]")
# med_81012.click()
# print(driver.page_source)

In [ ]:
# Search list of medicines
meds = driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]")
print(len(meds))
meds

25


[<selenium.webdriver.remote.webelement.WebElement (session="ef742d0380f32d78498d5ae119d1a8f5", element="9c50a6f5-cede-4502-b36d-a528be9f1b53")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ef742d0380f32d78498d5ae119d1a8f5", element="89e4f7e9-ea6d-4017-a9bc-bfe352be6bc4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ef742d0380f32d78498d5ae119d1a8f5", element="e59c1b0c-013e-4cda-b51f-d020941efb89")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ef742d0380f32d78498d5ae119d1a8f5", element="3abbcc45-92af-4fb6-8901-20fe4824c908")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ef742d0380f32d78498d5ae119d1a8f5", element="d2162356-bf47-4107-9b04-1e1f600a2eaf")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ef742d0380f32d78498d5ae119d1a8f5", element="648b3720-ac11-4fda-959a-d256ca3d3558")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ef742d0380f32d78498d5ae119d1a8f5", element="6f47dd0e-830f-454e-ae60-d6

In [ ]:
# Retrieve all ids within the list of medicines

meds_ids = []

for m in meds:
  id = m.get_attribute('onclick')
  meds_ids.append(id)

meds_ids

["medicamentoOnSelect('81012')",
 "medicamentoOnSelect('84182')",
 "medicamentoOnSelect('80803')",
 "medicamentoOnSelect('82913')",
 "medicamentoOnSelect('07428001')",
 "medicamentoOnSelect('65382')",
 "medicamentoOnSelect('65059')",
 "medicamentoOnSelect('61836')",
 "medicamentoOnSelect('61837')",
 "medicamentoOnSelect('60014')",
 "medicamentoOnSelect('70452')",
 "medicamentoOnSelect('70451')",
 "medicamentoOnSelect('69223')",
 "medicamentoOnSelect('71437')",
 "medicamentoOnSelect('79696')",
 "medicamentoOnSelect('62389')",
 "medicamentoOnSelect('66460')",
 "medicamentoOnSelect('66459')",
 "medicamentoOnSelect('67763')",
 "medicamentoOnSelect('66458')",
 "medicamentoOnSelect('67621')",
 "medicamentoOnSelect('67988')",
 "medicamentoOnSelect('67874')",
 "medicamentoOnSelect('64920')",
 "medicamentoOnSelect('64921')"]

In [ ]:
def scrape_data(html: str) -> pd.Series:
    #Utilizamos un objeto BeautifulSoup para scrapear la página
    bs = BeautifulSoup(html,"html.parser") 
    
    #A continuación se scrapean todos los elementos presentes en la página:
    
    #Nombre del medicamento: Está entre identificado por la tag h1
    medicamento = bs.find('h1').get_text()
    
    #Nombre del laboratorio: Lo identifica la id="nombrelabXS"
    laboratorio = bs.find('div', {'id':"nombrelabXS"}).get_text()
    
    #Número de registro:  Lo identifica la id="nregistroId"
    num_registro = bs.find('span', {'id': 'nregistroId'}).get_text()
    
    #Comprobamos si un medicamento se ha autorizado o no.
    #Esta información está contenida en la tag h2 identificada por el atributo id=estadoXS
    #Si no está autorizado, entonces la tag esta vacía. Si sí lo está entonces aparece el contenido no vacío con el formato "Autorizado ( dd/mm/aaaa )"
    autorizado_bool = not bs.find('h2', {'id': 'estadoXS'}).get_text() == ''
    if autorizado_bool: 
        autorizado_fecha = re.sub( 
        "\( | \)", #Quitamos los paréntesis y los espacios
        ""
        ,re.findall(
            "\(.*\)", #Encontramos mediante una expresión regular el contenido de la fecha 
            bs.find('h2', {'id': 'estadoXS'}).get_text())[0]
    ).strip() #Nos aseguramos de que no hay ningún espacio adicional 
    else : 
        autorizado_fecha = None
    
    #Explicación análoga al apartado anterior para, en este caso, el estado de suspendido 
    suspendido_bool = not bs.find('h2', {'id': 'estadoXSsec'}).get_text() == ''
    if suspendido_bool: 
        suspendido_fecha = re.sub( 
        "\( | \)",
        ""
        ,re.findall(
            "\(.*\)",
            bs.find('h2', {'id': 'estadoXSsec'}).get_text())[0]
    ).strip()
    else : 
        suspendido_fecha = None
    
    #Si el medicamento está o no comercializado se identifica con un tag h3 identificado por la id='estadocomercXS'
    comercializado_bool = not bs.find('h3', {'id': 'estadocomercXS'}) == None
    
    #Las siguientes columnas son listas que tienen todas la misma estructura: Una etiqueta div con una id que la identifica.  
    #Para extraer la información accedemos a la id correspondiente recorriendo todos los elementos de la lista (del html) y 
    #guardándola en una lista (de python)
    
    vias_administracion = bs.find('div', {'id':'viasadministracion'}).find_all('li')
    vias_administracion = [va.get_text() for va in vias_administracion]
    
    dosis = bs.find('div', {'id':'dosis'}).find_all('li')
    dosis = [d.get_text() for d in dosis]
    
    formas_farmaceuticas = bs.find('div', {'id':'formas'}).find_all('li')
    formas_farmaceuticas = [ff.get_text() for ff in formas_farmaceuticas]
    
    principios_activos = bs.find('div', {'id':"pactivosList"}).find_all('li')
    principios_activos = [pa.get_text() for pa in principios_activos]
    
    excipientes = bs.find('div', {'id':'excipientesList'}).find_all('li')
    excipientes = [e.get_text() for e in excipientes]
    
    caracteristicas = bs.find('div', {'id':'caracteristicasList'}).find_all('li')
    caracteristicas = [c.get_text() for c in caracteristicas]
    
    codigos_atc = bs.find('div', {'id':'atcList'}).find_all('li')
    codigos_atc = [atc.get_text() for atc in codigos_atc]
    
    #Se guardan todos los elementos extraídos anteriormente en una nueva fila cuyo índice en el DataFrame será el número de registro    
    nueva_fila = pd.Series(
        {
            'Número de registro':num_registro ,
            'Medicamento':medicamento,
            'Laboratorio':laboratorio,
            'Autorizado':autorizado_bool,
            'Fecha autorización': autorizado_fecha,
            'Suspendido': suspendido_bool,
            'Fecha suspensión': suspendido_fecha,
            'Comercializado': comercializado_bool,
            'Vías administración': vias_administracion,
            'Dosis': dosis,
            'Formas farmacéuticas': formas_farmaceuticas,
            'Principios activos': principios_activos,
            'Excipientes': excipientes,
            'Características':caracteristicas,
            'Códigos ATC': codigos_atc
        }
        , name=num_registro)  
    return nueva_fila

In [ ]:
tabla =pd.DataFrame([])
for i in meds_ids:  #Recorremos las ids de todos los medicamentos representado a cada uno por i 
    print(i)
    
    #Hacemos click en el elemento de la página web que se identifica por el atributo onclick=i que corresponde al medicamento
    #con el número de registro i  
    driver.find_element(By.CSS_SELECTOR,"div[onclick=\"{}\"]".format(i)).click() 
    
    #Esperamos hasta que se termine de cargar el contenido del html donde se encuentran todos los datos de interés del medicamento
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "figure"))) 
    
    #Accedemos al código fuente de la página una vez que esté se ha terminado de rellenar
    nueva_fila = scrape_data(html=driver.page_source)
    
    #Se añade la nueva fila a la tabla 
    tabla = tabla.append(nueva_fila)
    
    #Se vuelve atrás esperando a que la página cargue para seguir haciendo el mismo proceso para los demás medicamentos
    driver.back()
    driver.implicitly_wait(5)


medicamentoOnSelect('81012')
medicamentoOnSelect('84182')
medicamentoOnSelect('80803')
medicamentoOnSelect('82913')
medicamentoOnSelect('07428001')
medicamentoOnSelect('65382')
medicamentoOnSelect('65059')
medicamentoOnSelect('61836')
medicamentoOnSelect('61837')
medicamentoOnSelect('60014')
medicamentoOnSelect('70452')
medicamentoOnSelect('70451')
medicamentoOnSelect('69223')
medicamentoOnSelect('71437')
medicamentoOnSelect('79696')
medicamentoOnSelect('62389')
medicamentoOnSelect('66460')
medicamentoOnSelect('66459')
medicamentoOnSelect('67763')
medicamentoOnSelect('66458')
medicamentoOnSelect('67621')
medicamentoOnSelect('67988')
medicamentoOnSelect('67874')
medicamentoOnSelect('64920')
medicamentoOnSelect('64921')


In [ ]:
tabla

,Autorizado,Características,Comercializado,Códigos ATC,Dosis,Excipientes,Fecha autorización,Fecha suspensión,Formas farmacéuticas,Laboratorio,Medicamento,Número de registro,Principios activos,Suspendido,Vías administración
81012,1.0,"[USO HOSPITALARIO, Medical prescription]",1.0,"[J05A - ANTIVIRALES DE ACCIÓN DIRECTA, J05AR -...",[600 mg/300 mg],"[CARBOXIMETILALMIDON SODICO TIPO A, AMARILLO A...",26/06/2016,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],Reddy Pharma Iberia S.A.,ABACAVIR/LAMIVUDINA DR. REDDYS 600 MG/300 MG C...,81012,"[ABACAVIR, LAMIVUDINA]",0.0,[VÍA ORAL]
84182,1.0,"[USO HOSPITALARIO, Medical prescription]",1.0,"[J05A - ANTIVIRALES DE ACCIÓN DIRECTA, J05AR -...",[600 mg/ 300 mg],[],30/05/2019,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],"Kern Pharma, S.L.",ABACAVIR/LAMIVUDINA KERN PHARMA 600 MG/300 MG ...,84182,"[ABACAVIR, LAMIVUDINA]",0.0,[VÍA ORAL]
80803,1.0,"[USO HOSPITALARIO, Medical prescription]",1.0,"[J05A - ANTIVIRALES DE ACCIÓN DIRECTA, J05AR -...",[600 mg/300 mg],[],22/09/2016,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],"Sandoz Farmaceutica, S.A.",ABACAVIR/LAMIVUDINA SANDOZ 600 MG/300 MG COMPR...,80803,"[ABACAVIR CLORHIDRATO, LAMIVUDINA]",0.0,[VÍA ORAL]
82913,1.0,"[USO HOSPITALARIO, Medical prescription]",1.0,"[J05A - ANTIVIRALES DE ACCIÓN DIRECTA, J05AR -...",[600 - REVISAR mg],"[CARBOXIMETILALMIDON SODICO TIPO A, AMARILLO A...",30/05/2018,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],"Laboratorio Stada, S.L.",ABACAVIR/LAMIVUDINA STADA 600 MG/300 MG COMPRI...,82913,"[ABACAVIR, LAMIVUDINA]",0.0,[VÍA ORAL]
07428001,1.0,"[USO HOSPITALARIO, Medical prescription, Drivi...",1.0,[L01C - ALCALOIDES DE PLANTAS Y OTROS PRODUCTO...,[100 mg],[],20/01/2009,None,[POLVO PARA DISPERSIÓN PARA PERFUSIÓN],Bristol-Myers Squibb Pharma Eeig,ABRAXANE 5 MG/ML POLVO PARA DISPERSION PARA PE...,07428001,[PACLITAXEL],0.0,[VÍA INTRAVENOSA]
65382,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,"[C10A - AGENTES MODIFICADORES DE LOS LÍPIDOS, ...",[10 mg ezetimiba],"[CROSCARMELOSA SODICA, LACTOSA MONOHIDRATO, LA...",05/05/2003,None,[COMPRIMIDO],"Organon Salud, S.L.",ABSORCOL 10 mg COMPRIMIDOS,65382,[EZETIMIBA],0.0,[VÍA ORAL]
65059,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,[M01A - PRODUCTOS ANTIINFLAMATORIOS Y ANTIRREU...,[8 mg lornoxicam],"[HIDROGENO CARBONATO DE SODIO, PROPILENGLICOL,...",10/11/2002,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],"Grünenthal Pharma, S.A.",ACABEL RAPID 8 mg COMPRIMIDOS RECUBIERTOS CON ...,65059,[LORNOXICAM],0.0,[VÍA ORAL]
61836,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,[M01A - PRODUCTOS ANTIINFLAMATORIOS Y ANTIRREU...,[4 mg lornoxicam],"[CROSCARMELOSA SODICA, LACTOSA]",30/04/1998,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],"Grünenthal Pharma, S.A.",ACABEL 4 mg COMPRIMIDOS RECUBIERTOS CON PELICULA,61836,[LORNOXICAM],0.0,[VÍA ORAL]
61837,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,[M01A - PRODUCTOS ANTIINFLAMATORIOS Y ANTIRREU...,[8 mg],"[CROSCARMELOSA SODICA, LACTOSA MONOHIDRATO]",30/04/1998,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],"Grünenthal Pharma, S.A.",ACABEL 8 mg COMPRIMIDOS RECUBIERTOS CON PELICULA,61837,[LORNOXICAM],0.0,[VÍA ORAL]
60014,1.0,[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA. TRA...,1.0,"[G04B - OTROS PRODUCTOS DE USO UROLÓGICO, INCL...",[1080 mg],[],30/06/1993,None,[COMPRIMIDO DE LIBERACIÓN PROLONGADA],"Ferrer Internacional, S.A.",ACALKA 1080 mg COMPRIMIDOS DE LIBERACION PROLO...,60014,[CITRATO POTASIO],0.0,[VÍA ORAL]


## Versión con scroll

In [ ]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get('https://cima.aemps.es/cima/publico/home.html')
#driver.find_element(By.CSS_SELECTOR,"button[onclick='biosimilaresTodos()']") # Example search in the homepage

# Navigate to "listado de búsqueda"
buscador = driver.find_element(By.ID, "inputbuscadorsimple")
buscador.clear()
buscador.send_keys("*")
buscador.send_keys(Keys.ENTER)
# driver.implicitly_wait(10) # If we do not wait, the `page_source` will not have time to change and it will be showed the previous page (the homepage in this case)
#print(driver.page_source)

In [ ]:
# Scroll down by a defined number of elements
# References: https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python

MAX_ELEMENTS = 10000 # Ha tardado 1 hora en obtener todos estos elementos.

while True:
  n_meds =len(driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]"))
  if n_meds >= MAX_ELEMENTS:
    break
  driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  driver.implicitly_wait(15)

n_meds

In [ ]:
# Scroll down by a defined number of scrolls
num_veces_scroll = 5
print("Number of medicines found:")
for i in range(num_veces_scroll): 
    print(len(driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]")))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(4)

Number of medicines found:
25
50
75
100
125


In [ ]:
meds = driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]")
meds_ids = [] 
for m in meds:
    num_registro = m.get_attribute('onclick')
    num_registro = re.search("\d+",num_registro).group(0)
    meds_ids.append(num_registro)

#driver.close()
len(meds_ids)

In [ ]:
tabla =pd.DataFrame([])
for i in meds_ids:
    print('https://cima.aemps.es/cima/publico/detalle.html?nregistro={}'.format(i))
    
    driver = webdriver.Chrome()
    driver.get('https://cima.aemps.es/cima/publico/detalle.html?nregistro={}'.format(i))
    
    #Esperamos hasta que se termine de cargar el contenido del html donde se encuentran todos los datos de interés del medicamento
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "figure"))) 
    
    #Accedemos al código fuente de la página una vez que esté se ha terminado de rellenar
    nueva_fila = scrape_data(html=driver.page_source)
    
    #Se añade la nueva fila a la tabla 
    tabla = tabla.append(nueva_fila)
    driver.close()


meds_ids

In [ ]:
# Scroll down 

from selenium.common.exceptions import NoSuchElementException

MAX_ELEMENTS = 100

num_veces_scroll = 2
treated_meds = set()
tabla =pd.DataFrame([])
while len(treated_meds) < MAX_ELEMENTS: 
    meds = driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]")
    meds = {re.search("\d+",m.get_attribute('onclick')).group(0) for m in meds}
    new_meds = meds - treated_meds
    if len(new_meds)==0:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.implicitly_wait(10)
        sleep(3)
        continue
    next_med = new_meds.pop()
    #print(f"New medicine id: {next_med}")

    while True:
        try:
          element = driver.find_element(By.CSS_SELECTOR, "div[onclick=\"medicamentoOnSelect('{}')\"]".format(next_med))
          print(element.text)
          
          #WebDriverWait(driver, 20).until(EC.element_to_be_clickable(element)) #.click() 
          driver.execute_script("arguments[0].click();", element) # Same as element.click(), but the latter raises an ElementClickInterceptedException
          
          #Esperamos hasta que se termine de cargar el contenido del html donde se encuentran todos los datos de interés del medicamento
          WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "figure"))) 
          
          nueva_fila = scrape_data(html=driver.page_source)
          
          #Se añade la nueva fila a la tabla 
          tabla = tabla.append(nueva_fila)
          treated_meds.add(next_med)

          #Se vuelve atrás esperando a que la página cargue para seguir haciendo el mismo proceso para los demás medicamentos
          driver.back()
          driver.implicitly_wait(5)   
          break
        except NoSuchElementException:
          driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
          driver.implicitly_wait(10)


ACARBOSA TECNIGEN 50 mg COMPRIMIDOS
ACETILCISTEINA CINFA 600 mg POLVO PARA SOLUCION ORAL EFG
ACECLOFENACO STADA 100 MG COMPRIMIDOS RECUBIERTOS CON PELICULA EFG
ACECLOFENACO CINFA 100 mg COMPRIMIDOS RECUBIERTOS CON PELICULA EFG
ACABEL 4 mg COMPRIMIDOS RECUBIERTOS CON PELICULA
ACALKA 1080 mg COMPRIMIDOS DE LIBERACION PROLONGADA
ACETILCISTEINA CINFA 600 mg COMPRIMIDOS EFERVESCENTES EFG
ACABEL 8 mg COMPRIMIDOS RECUBIERTOS CON PELICULA
ACETILCISTEINA SANDOZ CARE 600 mg COMPRIMIDOS EFERVESCENTES
ABACAVIR/LAMIVUDINA KERN PHARMA 600 MG/300 MG COMPRIMIDOS RECUBIERTOS CON PELICULA EFG
ACARBOSA TECNIGEN 100 mg COMPRIMIDOS
ACETILCISTEÍNA BEXAL 200 mg POLVO PARA SOLUCIÓN ORAL EFG
ABRAXANE 5 MG/ML POLVO PARA DISPERSION PARA PERFUSION
ACETILCISTEINA TARBIS 600 mg POLVO EFERVESCENTE EFG
ABACAVIR/LAMIVUDINA STADA 600 MG/300 MG COMPRIMIDOS RECUBIERTOS CON PELICULA EFG
ABACAVIR/LAMIVUDINA DR. REDDYS 600 MG/300 MG COMPRIMIDOS RECUBIERTOS CON PELICULA EFG
ACETILCISTEINA SANDOZ CARE 200 mg POLVO PARA SOLUCI

In [ ]:
tabla.shape

(100, 15)

In [ ]:
tabla.head()

,Autorizado,Características,Comercializado,Códigos ATC,Dosis,Excipientes,Fecha autorización,Fecha suspensión,Formas farmacéuticas,Laboratorio,Medicamento,Número de registro,Principios activos,Suspendido,Vías administración
70451,1.0,[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA. TRA...,1.0,[A10B - FÁRMACOS HIPOGLUCEMIANTES EXCLUYENDO I...,[50 mg],[ALMIDON DE MAIZ PREGELATINIZADO],04/02/2009,None,[COMPRIMIDO],"Tecnimede España Industria Farmaceutica, S.A.",ACARBOSA TECNIGEN 50 mg COMPRIMIDOS,70451,[ACARBOSA],0.0,[VÍA ORAL]
66458,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,"[R05C - EXPECTORANTES, EXCLUYENDO COMBINACIONE...",[600 mg],"[ASPARTAMO (E-951), MANITOL (E-421), AMARILLO ...",15/11/2004,None,[POLVO PARA SOLUCIÓN ORAL],"Laboratorios Cinfa, S.A.",ACETILCISTEINA CINFA 600 mg POLVO PARA SOLUCIO...,66458,[ACETILCISTEINA],0.0,[VÍA ORAL]
79696,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,[M01A - PRODUCTOS ANTIINFLAMATORIOS Y ANTIRREU...,[100 mg],"[CROSCARMELOSA SODICA, DIESTEARATO DE GLICEROL]",29/03/2015,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],"Laboratorio Stada, S.L.",ACECLOFENACO STADA 100 MG COMPRIMIDOS RECUBIER...,79696,[ACECLOFENACO],0.0,[VÍA ORAL]
69223,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,[M01A - PRODUCTOS ANTIINFLAMATORIOS Y ANTIRREU...,[100 mg],[CROSCARMELOSA SODICA],08/08/2007,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],"Laboratorios Cinfa, S.A.",ACECLOFENACO CINFA 100 mg COMPRIMIDOS RECUBIER...,69223,[ACECLOFENACO],0.0,[VÍA ORAL]
61836,1.0,"[MEDICAMENTO SUJETO A PRESCRIPCIÓN MÉDICA, Med...",1.0,[M01A - PRODUCTOS ANTIINFLAMATORIOS Y ANTIRREU...,[4 mg lornoxicam],"[CROSCARMELOSA SODICA, LACTOSA]",30/04/1998,None,[COMPRIMIDO RECUBIERTO CON PELÍCULA],"Grünenthal Pharma, S.A.",ACABEL 4 mg COMPRIMIDOS RECUBIERTOS CON PELICULA,61836,[LORNOXICAM],0.0,[VÍA ORAL]


# Problemas encontrados

## Selenium

### StaleElementReferenceException

Selenium no permite iterar sobre un listado de elementos encontrados porque el `id` del elemento cambia cada vez que se regresa la página de resultados de la búsqueda. 

Está explicado en el apartado de "**The Element is not Attached to the DOM"**  en la siguiente página: [https://www.selenium.dev/exceptions/#stale_element_reference](https://www.selenium.dev/exceptions/#stale_element_reference)*texto en cursiva*


In [ ]:
# Search list of medicines
driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]")

[<selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="b1a1df32-a801-4239-bd87-a11ed4d554a5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="35e5bbda-56d3-43ca-8606-aa43302a0a79")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="38599acc-f179-4091-b252-19f5cf3ba306")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="6709c6ce-1dff-40ef-815d-5e4d3d3f1b96")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="d3e0c429-1a47-416b-89b3-31177cc1c65d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="3c1823cb-225a-443b-9238-e40bc368b87a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="3a5a20ff-ca7c-4680-b24b-a0

In [ ]:
# Refresh page and perform the same search
driver.refresh()
driver.implicitly_wait(5)
driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]")

[<selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="dc618bc7-cb7a-448d-ab52-d758b9b076df")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="7dac3da3-ed71-48d3-aab4-4f80aa87ee6e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="6e25dc01-9d39-4ac9-9c02-3bf2c395c818")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="a3e9f86c-f414-4fa6-bf0b-fad52edd5fe5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="51e77872-dc3c-4c47-a094-4c0f6d975030")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="65cc798d-26b4-4046-a233-7228ec44c86a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="84381eceac0843338c0e2bb89edeb957", element="9e0c38a2-9846-4113-9b12-ef

Se puede apreciar como realizando la misma búsqueda sobre la misma página, la referencia (identificador del elemento) ha cambiado. Ejemplo: `element="b1a1df32-a801-4239-bd87-a11ed4d554a5"` vs `element="dc618bc7-cb7a-448d-ab52-d758b9b076df"`. Es por esto que al iterar sobre la primera búsqueda y volver atras, no podemos clicar/acceder al siguiente elemento (ya que el id será diferente)

In [ ]:
# Search list of medicines
meds = driver.find_elements(By.CSS_SELECTOR,"div[onclick*=medicamentoOnSelect]")
for m in meds:
  m.click() #La excepción salta en el momento de querer acceder al 2º elemento (al 1º sí que puede acceder)
  driver.implicitly_wait(5)
  driver.back()